In [1]:
import pandas as pd
import re
import json
import math
from itertools import permutations
from IPython.display import clear_output
from time import sleep
from abc import ABC, abstractmethod, abstractproperty

from grid_builder import *
from grid import *
from tile import *
from tiles import *
from destructable import *
from unit import *
from command import *
from ability import *
from shell import *
from beam import *
from game import *

In [2]:
gb = GridBuilder()

gb.place( WaterTile(),      (0,4) )
gb.place( WaterTile(),      (0,5) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (0,6) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (0,7) )

gb.place( Destructable(CorporateTile(), GroundTile(), 1), (1,1) )
gb.place( ForestTile(),     (1,2) )
gb.place( Destructable(CivilianTile(), GroundTile(), 1),  (1,3) )
gb.place( Destructable(CivilianTile(), GroundTile(), 1),  (1,4) )
gb.place( WaterTile(),      (1,5) )
gb.place( WaterTile(),      (1,6) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (1,7) )

gb.place( ForestTile(),     (2,3) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (2,5) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (2,6) )
gb.place( WaterTile(),      (2,7) )

gb.place( Destructable(CivilianTile(), GroundTile(), 1),  (3,1) )
gb.place( WaterTile(),      (3,7) )

gb.place( Destructable(CivilianTile(), GroundTile(), 2),  (4,1) )
gb.place( Destructable(CivilianTile(), GroundTile(), 2),  (4,5) )
gb.place( Destructable(CivilianTile(), GroundTile(), 2),  (4,6) )

gb.place( ForestFireTile(), (5,1) )
gb.place( ForestFireTile(), (5,2) )
gb.place( SpawnTile(),      (5,4) )
gb.place( ForestTile(),     (5,7) )

gb.place( Destructable(MountainTile(), GroundTile(), 3),   (6,0) )
gb.place( SpawnTile(),      (6,2) )
gb.place( ForestFireTile(), (6,3) )
gb.place( ForestFireTile(), (6,4) )

gb.place( Destructable(MountainTile(), GroundTile(), 3),   (7,0) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (7,1) )
gb.place( ForestTile(),     (7,3) )
gb.place( Destructable(MountainTile(), GroundTile(), 3),   (7,7) )

gb.place_on_tile( Mech("Siege Mech"),     (3,2) )
gb.place_on_tile( Mech("Artillery Mech"), (5,3) )
gb.place_on_tile( Mech("Boulder Mech"),   (7,3) )
gb.place_on_tile( Vek("Psy"),             (4,2) )
gb.place_on_tile( Vek("Alpha Firefly"),   (4,4) )
gb.place_on_tile( Vek("Scarab"),          (6,1) )
gb.place_on_tile( Vek("Firefly"),         (6,5) )
gb.place_on_tile( Unit("Boulder", max_health=1, health=1, moves=0), (7,5) )

grid = gb.to_grid()

for mech in grid.mechs:
    mech.add(Move(mech,grid))
    mech.add(Repair(mech))
    
for vek in grid.veks:
    vek.add(Move(vek, grid))

    
siege_mech     = grid.units['Siege Mech']
artillery_mech = grid.units['Artillery Mech']
boulder_mech   = grid.units['Boulder Mech']

firefly        = grid.units['Firefly']
alpha_firefly  = grid.units['Alpha Firefly']
scarab         = grid.units['Scarab']
psy            = grid.units['Psy']


siege_mech.add(Artillery(siege_mech, grid, ClusterShell, 2))
boulder_mech.add(Artillery(boulder_mech, grid, BoulderShell, 2))
artillery_mech.add(Artillery(artillery_mech, grid, RegularShell, 1))
    
grid.show()

,0,1,2,3,4,5,6,7
0,🟢,🟢,🟢,🟢,💦,💦,🗻,🗻
1,🟢,🏢 ϟ,🌲,🏘️ ϟ,🏘️ ϟ,💦,💦,🗻
2,🟢,🟢,🟢,🌲,🟢,🗻,🗻,💦
3,🟢,🏘️ ϟ,Siege Mech ♡♡♡♡,🟢,🟢,🟢,🟢,💦
4,🟢,🏘️ ϟϟ,Psy ♡♡♡♡,🟢,Alpha Firefly ♡♡♡♡,🏘️ ϟϟ,🏘️ ϟϟ,🟢
5,🟢,🔥,🔥,Artillery Mech ♡♡♡♡,⬆️,🟢,🟢,🌲
6,🗻,Scarab ♡♡♡♡,⬆️,🔥,🔥,Firefly ♡♡♡♡,🟢,🟢
7,🗻,🗻,🟢,Boulder Mech ♡♡♡♡,🟢,Boulder ♡,🟢,🗻


In [3]:
class Executor:
    def __init__(self):
        self.history = []
        
    def execute(self, command):
        self.history.append(command)
        command.execute()
        
    def undo(self):
        command = self.history.pop()
        command.undo()

In [4]:
class DFS:
    @staticmethod
    def rate(grid):
        power = sum(sum(tile.type_object.health for tile in tiles if '🏢' in repr(tile) or '🏘️' in repr(tile)) for tiles in grid.tiles)
        mech_alive = sum(1 for mech in grid.mechs if mech.health > 0)
        mech_total = sum(mech.health for mech in grid.mechs)
        vek_alive = sum(1 for vek in grid.veks if vek.health > 0)
        vek_total = sum(vek.health for vek in grid.veks)
        return {'Power':power, 'Veks':vek_alive, 'Vek Total Health':vek_total, 'Mechs':mech_alive, 'Mech Total Health':mech_total}
    
    
    def __init__(self, grid):
        self.grid = grid
        self.ex = Executor()
        self.explored = {}
        self.current = ""
        self.search()
        
    def order(self, action):
        rating_before = self.rate(grid)
        action.execute()
        rating_after = self.rate(grid)
        action.undo()
        score = 0
        
        score += (rating_after['Power'] - rating_before['Power'])*5
        score += rating_before['Vek Total Health'] - rating_after['Vek Total Health']
        score += (rating_before['Veks'] - rating_after['Veks'])*10
        score += (rating_after['Mechs'] - rating_before['Mechs'])*10
        score += rating_after['Mech Total Health'] - rating_before['Mech Total Health']
        
        return score
        
        
    def gen_frontier(self):
        frontier = []
        for mech in self.grid.mechs:
            for k,action in mech.gen_actions().items():
                frontier.append(action)
        return frontier
    
    # now go through all options for 1 round
    def search(self):
        frontier = self.gen_frontier()
        if not frontier:
            score = self.rate(self.grid)
            print(self.current, score)
            self.explored[self.current] = self.rate(self.grid)
            return
        
        frontier.sort(key=self.order, reverse=True)

        for i,action in enumerate(frontier):
            char = str(i) + '_'
            self.current += char
            action.execute()
            self.search()
            action.undo()
            self.current = self.current[:-len(char)]
            
    def __repr__(self):
        string = ""
        for i,v in self.explored.items():
            string += f"{i}) {v}"
        return string

In [5]:
DFS(grid)

0_0_0_0_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_0_0_1_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_0_0_2_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_0_0_3_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_0_0_4_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_0_0_5_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_0_0_6_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_0_0_7_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_0_0_8_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 10}
0_0_0_9_ {'Power': 9, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_0_0_10_ {'Power': 9, 'Veks': 3, 'Vek Total Health

0_1_5_2_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 12}
0_1_5_3_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 12}
0_1_5_4_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 12}
0_1_5_5_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 12}
0_1_5_6_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 12}
0_1_5_7_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 12}
0_1_5_8_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 12}
0_1_5_9_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 12}
0_1_5_10_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 12}
0_1_5_11_ {'Power': 9, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 12}
0_1_6_0_ {'Power': 10, 'Veks': 3, 'Vek Total Heal

0_2_0_6_2_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 12}
0_2_0_6_3_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 12}
0_2_0_6_4_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 12}
0_2_0_6_5_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 12}
0_2_0_6_6_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 12}
0_2_0_6_7_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 12}
0_2_0_6_8_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 12}
0_2_0_6_9_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 10}
0_2_0_6_10_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 10}
0_2_0_7_0_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 12}
0_2_0_7_1_ {'Power': 10, 'Vek

0_2_1_7_0_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_2_1_7_1_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_2_1_7_2_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_2_1_7_3_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_2_1_7_4_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_2_1_7_5_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_2_1_7_6_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_2_1_7_7_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_2_1_7_8_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_2_1_7_9_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_2_1_7_10_ {'Power': 10, 'Vek

0_2_2_7_7_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_2_2_7_8_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_2_2_7_9_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_2_2_7_10_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_2_2_7_11_ {'Power': 8, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_2_2_8_0_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_2_2_8_1_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_2_2_8_2_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_2_2_8_3_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 11}
0_2_2_8_4_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_2_2_8_5_ {'Power': 10, 'Vek

...

0_8_2_1_0_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_2_1_1_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_2_1_2_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_2_1_3_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_2_1_4_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_2_1_5_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_2_1_6_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_2_1_7_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 11}
0_8_2_1_8_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_2_1_9_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_2_1_10_ {'Power': 9, 'Veks

0_8_3_2_0_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_3_2_1_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_3_2_2_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_3_2_3_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_3_2_4_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_3_2_5_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_3_2_6_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 11, 'Mechs': 3, 'Mech Total Health': 11}
0_8_3_2_7_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_3_2_8_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_3_2_9_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 10}
0_8_3_2_10_ {'Power': 9, 'Veks

0_8_4_5_3_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_4_5_4_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_4_5_5_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_4_5_6_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_4_5_7_ {'Power': 9, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_4_5_8_ {'Power': 8, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_4_5_9_ {'Power': 8, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_4_5_10_ {'Power': 8, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_4_6_0_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 10, 'Mechs': 3, 'Mech Total Health': 12}
0_8_4_6_1_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_4_6_2_ {'Power': 10, 'Veks': 

0_8_5_7_10_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 10}
0_8_5_8_0_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 10, 'Mechs': 3, 'Mech Total Health': 12}
0_8_5_8_1_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_5_8_2_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_5_8_3_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_5_8_4_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_5_8_5_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_5_8_6_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_5_8_7_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_5_8_8_ {'Power': 10, 'Veks': 3, 'Vek Total Health': 12, 'Mechs': 3, 'Mech Total Health': 12}
0_8_5_8_9_ {'Power': 10, 'Vek

KeyboardInterrupt: 

In [ ]:
grid.show()

In [ ]:
stop

In [ ]:
frontier = dfs.gen_frontier()

In [ ]:
len(frontier)

In [ ]:
frontier

In [ ]:
frontier[5].execute()

In [ ]:
frontier2 = dfs.gen_frontier()

In [ ]:
len(frontier2)

In [ ]:
frontier2[0].execute()

In [ ]:
frontier3 = dfs.gen_frontier()

In [ ]:
len(frontier3)

In [ ]:
frontier3[0].execute()

In [ ]:
frontier4 = dfs.gen_frontier()

In [ ]:
len(frontier4)

In [ ]:
frontier4[5].execute()

In [ ]:
frontier5 = dfs.gen_frontier()

In [ ]:
len(frontier5)

In [ ]:
frontier5[0].execute()

In [ ]:
frontier6 = dfs.gen_frontier()

In [ ]:
len(frontier6)

In [ ]:
frontier6[2].execute()

In [ ]:
frontier7 = dfs.gen_frontier()

In [ ]:
frontier7

In [ ]:
"1111111"

In [ ]:
grid.show()

In [ ]:
DFS(grid)